# Inspect/Visualise CNFM-E fit to motion corrected data

In [ ]:
try:
    get_ipython().magic(u'load_ext autoreload')
    get_ipython().magic(u'autoreload 2')
    get_ipython().magic(u'matplotlib qt')
except:
    pass

import matplotlib.pyplot as plt
import numpy as np
import os
from tqdm import tqdm
import napari


import caiman as cm
from caiman.source_extraction.cnmf.spatial import threshold_components
from caiman.utils.visualization import inspect_correlation_pnr

import logging
from fancylog import fancylog
import fancylog as package

import cv2
try:
    cv2.setNumThreads(0)
except:
    pass
import bokeh.plotting as bpl

from fcutils.file_io.io import load_yaml
from fcutils.video.utils import open_cvwriter, get_cap_from_images_folder, save_videocap_to_video
from fcutils.plotting.utils import clean_axes, save_figure, add_colorbar_to_img

from utils import print_cnmfe_components, plot_components_over_image, load_fit_cnmfe
from utils import start_server, load_params,  log_cnmfe_components, load_mmap_video_caiman

bpl.output_notebook()
c, dview, n_processes = start_server()

## Load metadata, model and data

In [ ]:
fld = "D:\\Dropbox (UCL - SWC)\\Project_vgatPAG\\analysis\\doric\\BF164p1\\19JUN05"
metadata = load_yaml(os.path.join(fld, "01_PARAMS", "analysis_metadata.yml"))

# Load video
video = os.path.join(fld, metadata[metadata['video_for_cnmfe_fit']])
Yr, dims, T, images = load_mmap_video_caiman(video)

# Load model
cnm, model_filepath = load_fit_cnmfe(fld, n_processes, dview)


# Get filtered and peak over noise ratio images
cn_filter, pnr = cm.summary_images.correlation_pnr(images, gSig=cnm.params.init['gSig'][0], swap_dim=False)

# Start logging
fancylog.start_logging(os.path.join(fld, "02_LOGS"), package, file_log_level="INFO", variables=[cnm.params], verbose=True,    filename='cnmfe_component_inspection_logs')
logging.info("Starting CNMF-E component inspection")

# Add analysis metadata to log
logging.info("ANALYSIS METADATA FILE:")
for k,v in metadata.items():
    logging.info(f"{k}: {v}")

logging.info(f"Video used for CNMF-E fitting: {metadata['video_for_cnmfe_fit']}: {video}")


# Inspect components

Apply a number of quality controls of the components to keep only the good stuff.
Quality control steps include:

    - evaluate components 
    - threshold spatial components -> remove duplicates
    - remove small/large components [currently not implemented]

there's no undo for these operations, so if you need to restart from scratch you'll need
to re load cnm:

    cnm = load_fit_cnfm(model_path, n_processes, dview)

To check the params used for quality check use:

    cnm.params.quality

In [ ]:
# Lets look at what things are like to start with
cnm.params.quality
print_cnmfe_components(cnm, msg="Before quality control:")
log_cnmfe_components(cnm,  msg="Before quality control")


# Estimate
cnm.estimates.evaluate_components(images, cnm.params, dview=dview
coordinates = cm.utils.visualization.get_contours(cnm.estimates.A, cn_filter.shape, thr=.2, thr_method="max")
good_compontents = cnm.estimates.idx_components)

# log
print_cnmfe_components(cnm, msg="After evaluate components")
log_cnmfe_components(cnm,  msg="After evaluate components")

### Remove overlapping components

In [ ]:
# Estimate spatial components
cnm.estimates.dims = dims
cnm.estimates.threshold_spatial_components(dview=dview)


if metadata['components_qc_params']['filter_by_size']:
    logging.info("Filtering components based on size")

    # load params
    _params = load_yaml(os.path.join(fld, "01_params", "params.yml"))['quality_evaluation_params']

    # Remove large/small and duplicate neurons
    cnm.estimates.remove_small_large_neurons(_params['min_size_neuro'], _params['max_size_neuro'])
    print_cnmfe_components(cnm, msg="After filter by size")
    log_cnmfe_components(cnm,  msg="After filter by size")

if metadata['components_qc_params']['remove_duplicate_components']:
    logging.info("Removing duplicates")

    _ = cnm.estimates.remove_duplicates(plot_duplicates=False)

    print_cnmfe_components(cnm, msg="After Remove overlapping")
    log_cnmfe_components(cnm,  msg="After Remove overlapping")

coordinates = cm.utils.visualization.get_contours(cnm.estimates.A, cn_filter.shape, thr=.2, thr_method="max")
good_compontents = cnm.estimates.idx_components

## Visualise Components location

Visualise the location of good and bad components on the cn_filt image and the residual optic flow image

In [ ]:
# Visualise good/bad components over the cn_filter image
f, axarr = plt.subplots(figsize=(15, 10), ncols=2)
for ax, only in zip(axarr, ["good", "bad"]):
    plot_components_over_image(cn_filter, ax, coordinates, 2, good_compontents, cmap="gray", only=only)
axarr[0].set(title="GOOD components")
axarr[1].set(title="BAD components")
clean_axes(f)

img_filepath = os.path.join(fld, "components_not_curated")
save_figure(f, img_filepath)
logging.info(f"Saved image with contours at {img_filepath}")

In [ ]:
# Visualise good/bad components over the residual optic flow image

optflow_img_path = os.path.join(metadata['fld'], metadata['outputfld'], 'residual_opticflow_raw.npy')
optflow_img = np.load(optflow_img_path)

f, axarr = plt.subplots(figsize=(15, 10), ncols=2)
for ax, only in zip(axarr, ["good", "bad"]):
    img = plot_components_over_image(optflow_img, ax, coordinates, 2, good_compontents, cmap="viridis", only=only)
    add_colorbar_to_img(img, ax, f)

axarr[0].set(title="GOOD components")
axarr[1].set(title="BAD components")
clean_axes(f)

img_filepath = os.path.join(fld, "components_not_curated_on_optflow")
save_figure(f, img_filepath)
logging.info(f"Saved image with contours at {img_filepath}")


# MANUAL COMPONENTS CURATION
TODO...

In [ ]:
f, axarr = plt.subplots(figsize=(15, 10), ncols=2)
for ax, im, ttl in zip(axarr, [cn_filter, cn_filter], ["good", "bad"]):
    plot_components_over_image(im, ax, coordinates, 2, good_compontents, cmap="gray", only=ttl)
axarr[0].set(title="GOOD components")
axarr[1].set(title="BAD components")
clean_axes(f)

img_filepath = os.path.join(fld, "components_curated")
save_figure(f, img_filepath)
logging.info(f"Saved image with contours at {img_filepath}")

## Save updated cnmfe model
If you're happy with the results, you can save the model for further analysis


In [ ]:
new_model_path = os.path.join(fld, "cnmfe_fit_curated.hdf5")
cnm.save(new_model_path)
logging.info(f"Saving updated model at {new_model_path}")

# Visualise components signal
Create plots and videos to visualise the location and the signals and components

In [ ]:

# if you want to look at bad components use: cnm.estimates.idx_components_bad instead of cnm.estimates.idx_components
cnm.estimates.nb_view_components(img=cn_filter, 
                                idx=cnm.estimates.idx_components,
                                denoised_color='red', 
                                cmap='gray')

## Make video with components over motion corrected data

Use the first cell if you just want to quickly look at the data, otherwise the next cell can be used to create a video but it's quite slow

In [ ]:
# To quickly look at the data use this code

# Get contours 
good_coords = [c['coordinates'][1:-2, ::-1] for c in coordinates if c['neuron_id'] in good_compontents]
good_coords = [c[~np.isnan(c).any(axis=1)] for c in good_coords] # remove nans
good_coords = [np.vstack([c, c[0]]) for c in good_coords]

bad_coords = [c['coordinates'][1:-2, ::-1] for c in coordinates if c['neuron_id'] not in good_compontents]
bad_coords = [c[~np.isnan(c).any(axis=1)] for c in bad_coords] # remove nans
bad_coords = [np.vstack([c, c[0]]) for c in bad_coords]

# Napari viewer
v = napari.Viewer(ndisplay=2)

v.add_image(images, name="Signal", colormap="gray", contrast_limits=[340, 600])

# add good components
layer = v.add_shapes(
    good_coords,
    shape_type='path',
    edge_width=0.5,
    edge_color='green',
    opacity=.5,
    face_color='royalblue',
    name='GOOD',
)

# add bad components
layer = v.add_shapes(
    bad_coords,
    shape_type='path',
    edge_width=0.15,
    edge_color='red',
    opacity=.3,
    face_color='royalblue',
    name='BAD',
)



In [ ]:
good_coords[0].shape

In [ ]:
frames_fld = "D:\\Dropbox (UCL - SWC)\\Project_vgatPAG\\analysis\\doric\\Fede\\frames" # folder where frames will be saved before creating video


tot_frames, w, h = images.shape
frames = np.arange(0, tot_frames, 10)

plt.ioff()
print(f"Creating video with {len(frames)} frames")

# Create an image for each frame and save it
if TrFalse  # <-- !! Set this as true if you want to create the video, keeping it False to avoid doing this by accident as it's slow   for n, fnum in tqdm(enumerate(frames)):
        f, ax = plt.subplots(figsize=(120, 100), dpi=5)
        plot_components_over_image(images[fnum, :, :].copy(), ax, coordinates, 17, good_compontents)
        f.savefig(os.path.join(frames_fld, f"{n}"), dpi=5)
        plt.close()
        
plt.ion()

# Stitch images into a video
""" 
Use this ffmpeg command to stitch the frames inot a video

first cd to the folder parent of the on where you saved the frames in. 
e.g. if you saved the frames here: D:\\Dropbox (UCL - SWC)\\Project_vgatPAG\\analysis\\doric\\Fede\\frames
then cd to: D:\\Dropbox (UCL - SWC)\\Project_vgatPAG\\analysis\\doric\\Fede

Then use:
ffmpeg -i frames\%1d.png -c:v libx264 -vf fps=10 -pix_fmt yuv420p out.mp4

the video will be saved as D:\\Dropbox (UCL - SWC)\\Project_vgatPAG\\analysis\\doric\\Fede\\out.mp4
"""

# Stop cluster

In [ ]:
cm.stop_server(dview=dview)